# Scratchpad notes for scraping NYS BOE contributions site

## Site being scraped
https://publicreporting.elections.ny.gov/Contributions/Contributions

## Scrapy Commands
### Start scrapy shell in terminal to explore above site
scrapy shell https://publicreporting.elections.ny.gov/Contributions/Contributions

### Xpath command notes while using above in terminal outside notebook
#### First mandatory field: Search by
response.xpath('/html/body/div[2]/section/form/div[2]/div[3]/div/div/div[5]/div[3]/select/option[2]/text()').getall()

returns ['Candidate']

for 2:5 in the number after `option`:
- `Candidate`
- `Commitee`
- `Contributor`
- `Office`

which is all the options, sweet


#### Options in mandatory field `office`

`Office` seems to have the smallest / least changing number of options

for completeness' sake, this is the xpath for the "search by office" option from above:
response.xpath('/html/body/div[2]/section/form/div[2]/div[3]/div/div/div[5]/div[3]/select/option[5]/text()').getall()


### `POST` Snag
Looks like there are `POST` requests to different sites for different drop-down toggles?

For example:
selecting `Office` in `Search by` field generates `POST` request to https://publicreporting.elections.ny.gov/Contributions/GetOfficeByOffType


selecting an office in the `Office` drop-down generates `POST` request to https://publicreporting.elections.ny.gov/Contributions/GetFilersByOffice

selecting an individual candidate in the `Candidate` drop-down generates `POST` request to https://publicreporting.elections.ny.gov/Contributions/GetCommitteeNameForAuthCand

clicking search generates two `POST` requests:
https://publicreporting.elections.ny.gov/Contributions/BindContributionsData
https://publicreporting.elections.ny.gov/Contributions/GetTotalContrAmount

The latter looks like it generates the total contributed amount which is in text above the resulting data table

So: try `POST`ing info to the `BindContributionsData` URL?

#### Breaking down step by step

First, selecting `Office` in the `Search by` field generates a `POST` response to https://publicreporting.elections.ny.gov/Contributions/GetOfficeByOffType

By copying the JSON of that `POST` request and feeding it back into https://publicreporting.elections.ny.gov/Contributions/GetOfficeByOffType, we get JSON of the options of offices to select

In scrapy shell:
```
url = 'https://publicreporting.elections.ny.gov/Contributions/GetOfficeByOffType'

#dictionary is copied JSON from inspector, GetOfficeByOffType
office_payload = {"lstUCOfficeType":"0","lstUCCounty":"","lstUCMuncipality":""}

req = FormRequest(url, formdata=office_payload)

fetch(req)

data = json.loads(response.body)
```

data is now a list of dictionaries of options to select for `Office`, eg

```
[{'Selected': False, 'Text': 'Alt Judicial Del. - State', 'Value': '17'},
 {'Selected': False, 'Text': 'Attorney General - State', 'Value': '7'},
 ...  
```

2021-03-17 19:17:46 [scrapy.utils.log] INFO: Scrapy 2.4.1 started (bot: scrapybot)
2021-03-17 19:17:46 [scrapy.utils.log] INFO: Versions: lxml 4.6.2.0, libxml2 2.9.10, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 21.2.0, Python 3.8.8 (default, Feb 24 2021, 13:46:16) - [Clang 10.0.0 ], pyOpenSSL 20.0.1 (OpenSSL 1.1.1j  16 Feb 2021), cryptography 3.4.6, Platform macOS-10.13.6-x86_64-i386-64bit
2021-03-17 19:17:46 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.selectreactor.SelectReactor
2021-03-17 19:17:46 [scrapy.crawler] INFO: Overridden settings:
{'DUPEFILTER_CLASS': 'scrapy.dupefilters.BaseDupeFilter',
 'LOGSTATS_INTERVAL': 0}
2021-03-17 19:17:46 [scrapy.extensions.telnet] INFO: Telnet Password: cd11b7629c6c733d
2021-03-17 19:17:46 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.memusage.MemoryUsage']
2021-03-17 19:17:46 [scrapy.middleware] INFO: Enabled downloade